# Multi-vector embeddings with ModernColBERT
This recipe explores how to use [LightOn's GTE-ModernColBERT-v1](https://huggingface.co/lightonai/GTE-ModernColBERT-v1) model to generate multi-vector embeddings for text data and use them in Weaviate. 

Multi-vector embeddings represent each object or query using multiple vectors instead of a single vector. This approach enables more precise searching through "late interaction", a technique that matches individual parts of texts rather than comparing them as whole units.

Note, this Notebook requires Weaviate `v1.30.1` abd  Weaviate client `v4.14.1`.

# Prerequisites
Before starting this tutorial, ensure you have the following:



In [ ]:
!pip install -U pylate
!pip install -U weaviate-client
!pip install -U sentence-transformers


# 1.1. Connect to Weaviate
First, connect to your Weaviate instance using your preferred client library. In this example, we assume you are connecting to a local Weaviate instance. For other types of instances, replace the connection details as needed (connection examples).

You can start a local Weaviate instance with this command:

In [2]:
!docker run --detach -p 8080:8080 -p 50051:50051 cr.weaviate.io/semitechnologies/weaviate:1.30.1

7deacc244c58eb5a3b10f9fba098fdbe17f970fe0258ab5c9f69882fb74f629a


Then connect to your local Weaviate instance.

In [4]:
import weaviate

# Option 1: Connect to your local Weaviate instance deployed with Docker
client = weaviate.connect_to_local()

# Option 2: Connet to an embedded Weaviate instance
# client = weaviate.connect_to_embedded()

# Option 3: Connect to your Weaviate Client Service cluster
# client = weaviate.connect_to_wcs(
#     cluster_id="WCS-CLUSTER-ID", # Replace with your WCS cluster ID
#     auth_credentials=weaviate.AuthApiKey(
#       api_key="WCS-API-KEY" # Replace with your WCS API KEY
#     ),
# )

client.is_ready()

True

# Define collection
Next, we define a collection called "DemoCollection". Note that we do not use a model integration, as we will provide the embeddings manually.

In [5]:
from weaviate.classes.config import Configure, Property, DataType
from weaviate.util import generate_uuid5
from weaviate.classes.config import Configure

collection_name = "DemoCollection"

# Check if collection exists before deleting
if client.collections.exists(collection_name):
    client.collections.delete(collection_name)  # THIS WILL DELETE THE SPECIFIED COLLECTION AND ALL ITS OBJECTS
    
client.collections.create(
    collection_name,
    vectorizer_config=[
        # User-provided embeddings
        Configure.NamedVectors.none(
            name="multi_vector",
            vector_index_config=Configure.VectorIndex.hnsw(
                # Enable multi-vector index with default settings
                multi_vector=Configure.VectorIndex.MultiVector.multi_vector()
            )
        ),
    ],
    properties=[
        Property(name="text", 
                 data_type=DataType.TEXT, 
                 vectorize_property_name=False  # Explicitly disable property name vectorization
                 ),
        Property(name="docid", 
                 data_type=DataType.TEXT,
                 vectorize_property_name=False  # Explicitly disable property name vectorization
                 ),
    ],
)

You can double-check that you're using the MaxSim operator for the multi-vector embeddings.

In [6]:
import json

# Get collection
collection = client.collections.get(collection_name)

config = collection.config.get().vector_config['multi_vector'].vector_index_config

print(json.dumps(config.__dict__, indent=2, default=lambda o: o.__dict__ if hasattr(o, '__dict__') else str(o)))


{
  "multi_vector": {
    "aggregation": "maxSim"
  },
  "quantizer": null,
  "cleanup_interval_seconds": 300,
  "distance_metric": "cosine",
  "dynamic_ef_min": 100,
  "dynamic_ef_max": 500,
  "dynamic_ef_factor": 8,
  "ef": -1,
  "ef_construction": 128,
  "filter_strategy": "sweeping",
  "flat_search_cutoff": 40000,
  "max_connections": 32,
  "skip": false,
  "vector_cache_max_objects": 1000000000000
}


# 1.3. Import data

Now, we can import the data. For this example, we will import a few arbitrary text objects.

Note that in this example, each object is sent to Weaviate along with the corresponding multi-vector embedding. In the example, we obtain LightOn's ModernColBERT embeddings, but it could be any multi-vector embeddings.

Load ModernColBERT embedding model:

In [7]:
from pylate import models

# Load the ModernColBERT model
model = models.ColBERT(
    model_name_or_path="lightonai/GTE-ModernColBERT-v1",
)

/Users/leonie/Documents/code/recipes/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/leonie/Documents/code/recipes/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# An example dataset
documents = [
    {"id": "doc1", "text": "Weaviate is a vector database that is great for AI app builders."},
    {"id": "doc2", "text": "PyTorch is a deep learning framework that is great for AI model builders."},
    {"id": "doc3", "text": "For people building AI driven products, Weaviate is a good database for their tech stack."},
]


# Import data
with collection.batch.fixed_size(batch_size=10) as batch:
    for doc in documents:
        # Iterate through the dataset & add to batch
        batch.add_object(
            properties={"text": doc["text"], "docid": doc["id"]},
            uuid=generate_uuid5(doc["id"]),
            vector={"multi_vector": model.encode(doc["text"], is_query=False)},  # Provide the embedding manually
        )


In [9]:
# Check for errors in batch imports
if collection.batch.failed_objects:
    print(f"Number of failed imports: {len(collection.batch.failed_objects)}")
    print(f"First failed object: {collection.batch.failed_objects[0]}")

print(len(collection))  # This should print `3``

3


Let's retrieve an object and inspect the shape of its embeddings.

In [10]:
response = collection.query.fetch_objects(limit=3, include_vector=True)

for obj in response.objects:
    print(f"This embedding's shape is ({len(obj.vector['multi_vector'])}, {len(obj.vector['multi_vector'][0])})")


This embedding's shape is (17, 128)
This embedding's shape is (20, 128)
This embedding's shape is (18, 128)


Note this in contrast to a single vector, which would be a list of floats.



# 1.4. Perform vector search query
Now that we have imported the data, we can perform searches using the multi-vector embeddings. 

You can perform a manual vector search, by specifying the query embedding. In this example, we convert the query into a vector using the same model used to generate the object embeddings.

This ensures that the query embedding is compatible with the object embeddings.

In [11]:
query = "A good database for AI app builders"
response = collection.query.near_vector(
    near_vector=model.encode(query, is_query=True),  # Raw ColBERT embedding, in [[e11, e12, e13, ...], [e21, e22, e23, ...], ...] shape
    target_vector="multi_vector",
    return_metadata=weaviate.classes.query.MetadataQuery(
            distance=True,
        ),
)

for result in response.objects:
    print(result.properties)
    print(result.metadata.distance)


{'text': 'Weaviate is a vector database that is great for AI app builders.', 'docid': 'doc1'}
-29.469009399414062
{'text': 'For people building AI driven products, Weaviate is a good database for their tech stack.', 'docid': 'doc3'}
-29.417606353759766
{'text': 'PyTorch is a deep learning framework that is great for AI model builders.', 'docid': 'doc2'}
-28.900041580200195


# Additional resources
You might also enjoy the following resources:

- Tutorial: [Weaviate multi-vector embeddings](https://weaviate.io/developers/weaviate/tutorials/multi-vector-embeddings)
- Blog: [An Overview of Late Interaction Retrieval Models: ColBERT, ColPali, and ColQwen](https://weaviate.io/blog/late-interaction-overview)
- Recipe notebooks [on multi-vector embeddings](https://github.com/weaviate/recipes/tree/main/weaviate-features/multi-vector)

:::info
Multi-vector support is added in Weaviate v1.29 as a **technical preview**.

This means that the feature is still under development and may change in future releases, including potential breaking changes. Currently, quantization is not supported for multi-vector embeddings.

We do not recommend using this feature in production environments at this time.
:::